# PMF and virial coefficient calculation using VirtualTranslate

Let molecule A (index 0) be centered in the the middle of a spherical simulation cell and allowed to _rotate only_.
Place another molecule, B (index 1), in on the positive $z$-axis and let it translate (`dir=[0,0,1]`) and rotate in such a way that it
cannot cross to the negative $z$-axis. Add the following to the analysis:

~~~ yml
- virtualtranslate: {file: vt.dat, dL: 0.1, molecule: B, nstep: 10, dir: [0,0,1]}
- reactioncoordinate: {file: R.dat, nstep: 10, type: molecule, property: com_z, index: 1}
~~~

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
from scipy.stats import binned_statistic

def getForces(xfile, yfile, bins):
    ''' get average force as a function of z-position of molecule B '''
    R = np.loadtxt(xfile, usecols=[1])
    steps, dL, du, force = np.loadtxt(yfile, unpack=True, skiprows=1)
    means, edges, bins = binned_statistic(R, np.exp(-du), 'mean', bins)
    return edges[1:], (np.log(means) / dL[0])
    #return (edges[:-1] + edges[1:]) / 2, np.log(means) / dL[0]

## Run Faunus from bash

In [ ]:
%%bash
if [[ -z "${FAUNUS_EXECUTABLE}" ]]; then
  yason.py virial.yml | faunus --nobar
else
  echo "Seems we're running CTest - use Faunus target from CMake"
  "${YASON_EXECUTABLE}" virial.yml | "${FAUNUS_EXECUTABLE}" --nobar
fi

In [ ]:
# calculate mean force as a function of 
R, force = getForces('R.dat', 'vt.dat', 150)

In [ ]:
def lj(r, sigma=5, eps=0.40090549):
    return 4*eps*( (sigma/r)**12 - (sigma/r)**6  )

# LJ contribution to B2 from zero to contact
R0 = np.linspace(1e-6, R.min(), 500)
B2hs = -2 * np.pi * np.trapz( np.expm1( -lj(R0) )*R0**2, R0 )

# integrate simulated force to get PMF and normalize
pmf = -np.cumsum(force)*np.diff(R)[:-2].mean()
pmf = pmf - pmf[-1]

# integrate to get second virial
print("B2_hs = ", B2hs)
print("B2_lj = ", B2hs -2 * np.pi * np.trapz( np.expm1(-lj(R))*R**2, R )) 
print("B2_vt = ", B2hs -2 * np.pi * np.trapz( np.expm1(-pmf)*R**2, R ))

plt.plot(R, np.expm1(-pmf)*R**2, label='from force calc')
plt.plot(R, np.expm1(-lj(R))*R**2, label='LJ')
plt.legend(frameon=False, loc=0)